In [6]:
import os
import cv2
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split

# Constants
IMAGE_SIZE = 224
ORIGINAL_ANNOTATION_FILE = 'ObjectDetection.xlsx'  # Path to your original Excel file
IMAGE_DIR = 'Pre processed images'  # Directory containing original images
RESIZED_IMAGE_DIR = 'Resized_images'  # Directory where resized images will be saved
RESIZED_ANNOTATION_FILE = 'Resized_ObjectDetection.xlsx'  # Path to the new Excel file with updated annotations

# Create the resized images directory if it does not exist
if not os.path.exists(RESIZED_IMAGE_DIR):
    os.makedirs(RESIZED_IMAGE_DIR)

# Load original annotations
original_annotations = pd.read_excel(ORIGINAL_ANNOTATION_FILE)

# Function to scale annotation coordinates
def scale_annotations(row, orig_width, orig_height, target_size):
    x_scale = target_size / orig_width
    y_scale = target_size / orig_height
    row['h_min'] = int(row['h_min'] * y_scale)
    row['w_min'] = int(row['w_min'] * x_scale)
    row['h_max'] = int(row['h_max'] * y_scale)
    row['w_max'] = int(row['w_max'] * x_scale)
    return row

# Initialize a DataFrame to hold the resized annotations
resized_annotations = pd.DataFrame(columns=original_annotations.columns)

# Process each image and its annotations
for fname in original_annotations['fname'].unique():
    # Load the image
    image_path = os.path.join(IMAGE_DIR, f"{fname}.png")
    image = Image.open(image_path)
    orig_width, orig_height = image.size

    # Resize the image
    image = image.resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)
    
    # Save the resized image to the new directory
    resized_image_path = os.path.join(RESIZED_IMAGE_DIR, f"{fname}.png")
    image.save(resized_image_path)

    # Get annotations for this image and scale them
    image_annotations = original_annotations[original_annotations['fname'] == fname]
    scaled_annotations = image_annotations.apply(lambda row: scale_annotations(row, orig_width, orig_height, IMAGE_SIZE), axis=1)
    
    # Append the scaled annotations to the new DataFrame
    resized_annotations = pd.concat([resized_annotations, scaled_annotations], ignore_index=True)

# Save the resized annotations to a new Excel file
resized_annotations.to_excel(RESIZED_ANNOTATION_FILE, index=False)

print(f"All images have been resized and saved to {RESIZED_IMAGE_DIR}.")
print(f"Updated annotations have been saved to {RESIZED_ANNOTATION_FILE}.")


All images have been resized and saved to Resized_images.
Updated annotations have been saved to Resized_ObjectDetection.xlsx.
